# 0. Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/train.csv")
df
df = df.drop(columns = ["Id"])

In [3]:
df.shape

(1460, 80)

# 1.Preproccessing
## 1.1. Missing value

In [4]:
temp = df.isna().sum(axis = 0) 
temp[temp > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [5]:
df[temp[temp > 0].index].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   1201 non-null   float64
 1   Alley         91 non-null     object 
 2   MasVnrType    1452 non-null   object 
 3   MasVnrArea    1452 non-null   float64
 4   BsmtQual      1423 non-null   object 
 5   BsmtCond      1423 non-null   object 
 6   BsmtExposure  1422 non-null   object 
 7   BsmtFinType1  1423 non-null   object 
 8   BsmtFinType2  1422 non-null   object 
 9   Electrical    1459 non-null   object 
 10  FireplaceQu   770 non-null    object 
 11  GarageType    1379 non-null   object 
 12  GarageYrBlt   1379 non-null   float64
 13  GarageFinish  1379 non-null   object 
 14  GarageQual    1379 non-null   object 
 15  GarageCond    1379 non-null   object 
 16  PoolQC        7 non-null      object 
 17  Fence         281 non-null    object 
 18  MiscFeature   54 non-null   

Có 19 cột có missing value, chia thành 3 nhóm:
- Các cột numerical, với các cột này, nhóm chỉ điền giá trị mean vào missing value
- Các cột categorical có định nghĩa giá trị NA, với các cột này nhóm sẽ điền giá trị được định nghĩa trong mô tả dữ liệu vào missing value
- Các cột categorical không có định nghĩa giá trị NA, đối với các cột này, nhóm sẽ điền giá trị mode vào các missing value

In [6]:
#numerical features
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mean())
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['GarageYrBlt'].mean())

#categorical features defined NA
df['Alley'] = df['Alley'].fillna('No alley access')
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['BsmtQual'] = df['BsmtQual'].fillna('No basement')
df['BsmtCond'] = df['BsmtCond'].fillna('No basement')
df['BsmtExposure'] = df['BsmtExposure'].fillna('No basement')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('No basement')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('No basement')
df['FireplaceQu'] = df['FireplaceQu'].fillna('No fireplace')
df['GarageType'] = df['GarageType'].fillna('No garage')
df['GarageFinish'] = df['GarageFinish'].fillna('No garage')
df['GarageQual'] = df['GarageQual'].fillna('No garage')
df['GarageCond'] = df['GarageCond'].fillna('No garage')
df['PoolQC'] = df['PoolQC'].fillna('No pool')
df['Fence'] = df['Fence'].fillna('No fence')
df['MiscFeature'] = df['MiscFeature'].fillna('None')

#categorical features not defined NA
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

## 1.2. Duplicated values

In [7]:
df.duplicated().sum()

0

Dữ liệu không bị trùng

## 1.3. Outliers

Tại đây nhóm khảo sát dữ liệu theo 2 nhóm là categorical và numerical. Các cột có kiểu dữ liệu object được xếp vào categorical, ngược lại int và float được xếp vào numerical. Tuy nhiên cột MSSubClass có kiểu dữ liệu int nhưng các giá trị số nguyên này không có ý nghĩa so sánh lớn nhỏ mà chỉ có ý nghĩa phân loại, do đó nhóm sẽ gom cột này về categorical

### 1.3.1 Categorical outliers

In [8]:
dftypes = df.dtypes
categorical_col = list(dftypes[dftypes == "object"].keys())
categorical_col.append("MSSubClass")
categorical_col.append("KitchenAbvGr")

numerical_col = list(df.columns[~df.columns.isin(categorical_col)])[:-1]

In [9]:
# fig, axes = plt.subplots(10, 4, figsize = (20, 50))

# for col, ax in zip(categorical_col, axes.flatten()):
#     data = df[col].value_counts()
#     ax.barh(data.index, data.values)
#     ax.set_title(col)

Phân bố dữ liệu trong các cột categorical được chia thành 2 nhóm:

- Các cột dữ liệu phân bố đều, đối với các cột này, nhóm sẽ loại bỏ các dòng thuộc về không quá 5% dữ liệu
 
- Các cột dữ liệu chỉ tập trung vào một class, các class khác rất ít, sau khi thống nhất, nhóm quyết định vẫn giữ lại các cột này vì dù dữ liệu bị lệch cũng có thể đóng góp ít nhiều trong quá trình xây dựng mô hình

In [10]:
df1 = df

for col in categorical_col:
    temp = df1[col].value_counts()
    df1 = df1[~df1[col].isin(temp[temp <= 0.05 * df1.shape[0]].index)]

# fig, axes = plt.subplots(10, 4, figsize = (20, 50))

# for col, ax in zip(categorical_col, axes.flatten()):
#     data = df1[col].value_counts()
#     ax.barh(data.index, data.values)
#     ax.set_title(col)

df1.shape

(208, 80)

Khi thực hiện loại bỏ các giá trị thuộc về ít hơn 5% trong từng cột, nhóm nhận ra dữ liệu chỉ còn hơn 200 dòng. Do đó nhóm cũng sẽ giữ nguyên các cột này

### 1.3.2. Numerical outliers

In [11]:
# df[numerical_col].hist(bins = 40, figsize = (22.5, 15));

In [12]:
df1 = df

for col in numerical_col:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    df1 = df1[(df1[col] > (q1 - 1.5 * iqr)) & (df1[col] < (q3 + 1.5 * iqr))]

df1.shape

(0, 80)

Quan sát phân bố của các cột categorical và tiến hành loại bỏ thử các outliers sử dụng tứ phân vị, nhóm thu được kết quả tất cả điểm dữ liệu đều là outliers của ít nhất một feature nào đó. Do đó nhóm cũng sẽ không can thiệp vào các outliers ở các cột này

In [13]:
df.to_csv("../data/train_clean.csv", index = False)

Như vậy trong pha tiền xử lí, nhóm chỉ đơn giản điền các giá trị missing value cho các cột. Nhóm cũng sẽ tiền xử lí dữ liệu test tương tự ở bên dưới

In [14]:
df = pd.read_csv("../data/test.csv")

In [15]:
temp = df.isnull().sum(axis = 0)
temp[temp > 0].index

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')

In [16]:
#numerical features
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mean())
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['GarageYrBlt'].mean())
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(df['BsmtFinSF1'].mean())
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(df['BsmtFinSF2'].mean())
df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(df['BsmtUnfSF'].mean())
df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(df['TotalBsmtSF'].mean())
df['BsmtFullBath'] = df['BsmtFullBath'].fillna(df['BsmtFullBath'].mean())
df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(df['BsmtHalfBath'].mean())
df['GarageCars'] = df['GarageCars'].fillna(df['GarageCars'].mean())
df['GarageArea'] = df['GarageArea'].fillna(df['GarageArea'].mean())

#categorical features defined NA
df['Alley'] = df['Alley'].fillna('No alley access')
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['BsmtQual'] = df['BsmtQual'].fillna('No basement')
df['BsmtCond'] = df['BsmtCond'].fillna('No basement')
df['BsmtExposure'] = df['BsmtExposure'].fillna('No basement')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('No basement')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('No basement')
df['FireplaceQu'] = df['FireplaceQu'].fillna('No fireplace')
df['GarageType'] = df['GarageType'].fillna('No garage')
df['GarageFinish'] = df['GarageFinish'].fillna('No garage')
df['GarageQual'] = df['GarageQual'].fillna('No garage')
df['GarageCond'] = df['GarageCond'].fillna('No garage')
df['PoolQC'] = df['PoolQC'].fillna('No pool')
df['Fence'] = df['Fence'].fillna('No fence')
df['MiscFeature'] = df['MiscFeature'].fillna('None')

#categorical features not defined NA
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])
df['Utilities'] = df['Utilities'].fillna(df['Utilities'].mode()[0])
df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])
df['Functional'] = df['Functional'].fillna(df['Functional'].mode()[0])

In [17]:
df.to_csv("../data/test_clean.csv", index = False)